In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    Trainer,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [2]:
model_name = "mistralai/Mistral-7B-Instruct-v0.3"

dataset_name = "kokujin/prompts_1"

new_model = "Mistral-7B-Instruct-v0.3-luis"

In [3]:
# LoRA attention dimension
lora_r = 64

# Alpha parameter for LoRA scaling
lora_alpha = 16

# Dropout probability for LoRA layers
lora_dropout = 0.1

In [4]:
# Activate 8-bit precision base model loading
use_8bit = True

# Compute dtype for 4-bit base models
bnb_8bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = False

In [5]:
# Output directory where the model predictions and checkpoints will be stored
output_dir = f"./results/{model_name}/"

# Number of training epochs
num_train_epochs = 2

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = False

# Batch size per GPU for training
per_device_train_batch_size = 1

# Batch size per GPU for evaluation
per_device_eval_batch_size = 1

# Number of update steps to accumulate the gradients for
gradient_accumulation_steps = 3

# Enable gradient checkpointing
gradient_checkpointing = True

# Maximum gradient normal (gradient clipping)
max_grad_norm = 0.3

# Initial learning rate (AdamW optimizer)
learning_rate = 2e-4

# Weight decay to apply to all layers except bias/LayerNorm weights
weight_decay = 0.001

# Optimizer to use
optim = "paged_adamw_32bit"

# Learning rate schedule
lr_scheduler_type = "cosine"

# Number of training steps (overrides num_train_epochs)
max_steps = -1

# Ratio of steps for a linear warmup (from 0 to learning rate)
warmup_ratio = 0.03

# Group sequences into batches with same length
# Saves memory and speeds up training considerably
group_by_length = True

# Save checkpoint every X updates steps
save_steps = 500

# Log every X updates steps
logging_steps = 25

In [6]:
# Maximum sequence length to use
max_seq_length = 1000

# Pack multiple short examples in the same input sequence to increase efficiency
packing = False

# Load the entire model on the GPU 0
device_map = {"": 0}

In [7]:
# Load dataset (you can process it here)
dataset = load_dataset(dataset_name, split="train")

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_8bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_8bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_8bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

Your GPU supports bfloat16: accelerate training with bf16=True


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="Text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


In [9]:
# Train model
trainer.train(resume_from_checkpoint = False)

  0%|          | 0/6396 [00:00<?, ?it/s]

{'loss': 1.0361, 'grad_norm': 0.3715257942676544, 'learning_rate': 2.604166666666667e-05, 'epoch': 0.01}
{'loss': 0.8629, 'grad_norm': 0.9048253893852234, 'learning_rate': 5.208333333333334e-05, 'epoch': 0.02}
{'loss': 0.3976, 'grad_norm': 0.626203715801239, 'learning_rate': 7.8125e-05, 'epoch': 0.02}
{'loss': 0.272, 'grad_norm': 1.2425174713134766, 'learning_rate': 0.00010416666666666667, 'epoch': 0.03}
{'loss': 0.204, 'grad_norm': 0.34642747044563293, 'learning_rate': 0.00013020833333333333, 'epoch': 0.04}
{'loss': 0.1951, 'grad_norm': 0.9093351364135742, 'learning_rate': 0.00015625, 'epoch': 0.05}
{'loss': 0.1572, 'grad_norm': 0.2559562921524048, 'learning_rate': 0.00018229166666666667, 'epoch': 0.05}
{'loss': 0.2016, 'grad_norm': 0.42944344878196716, 'learning_rate': 0.00019999917944905216, 'epoch': 0.06}
{'loss': 0.1573, 'grad_norm': 0.23466379940509796, 'learning_rate': 0.00019998603811858571, 'epoch': 0.07}
{'loss': 0.1921, 'grad_norm': 0.3496451675891876, 'learning_rate': 0.000

TrainOutput(global_step=6396, training_loss=0.0634373522703166, metrics={'train_runtime': 30004.4917, 'train_samples_per_second': 0.64, 'train_steps_per_second': 0.213, 'total_flos': 8.049339626815488e+17, 'train_loss': 0.0634373522703166, 'epoch': 1.9997915581031789})

In [10]:
# Save trained model
trainer.model.save_pretrained(new_model)

In [11]:
tokenizer.save_pretrained(new_model + "_tokenizer")

('Mistral-7B-Instruct-v0.3-luis_tokenizer/tokenizer_config.json',
 'Mistral-7B-Instruct-v0.3-luis_tokenizer/special_tokens_map.json',
 'Mistral-7B-Instruct-v0.3-luis_tokenizer/tokenizer.model',
 'Mistral-7B-Instruct-v0.3-luis_tokenizer/added_tokens.json',
 'Mistral-7B-Instruct-v0.3-luis_tokenizer/tokenizer.json')